In [1]:
import os
import cx_Oracle
import pandas as pd
import numpy as np
import pickle
from datetime import date,datetime
from sklearn import metrics
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, cross_val_score 
import time
import matplotlib.pyplot as plt
from sklearn.utils import resample
import datetime

In [3]:
os.chdir(r"D:\HomeCredit\Bigdata\Common Maps KNN")

# Generate grid based scores for selected variables

### Query for Data Download

In [ ]:
# drop table bd_geo_delinq;
# create table bd_geo_delinq as
# select * from
# (
# select a.skp_credit_case, a.DATE_decision, a.latitude as cus_lat, a.longitude as cus_long, grid_id,
# b.RISK_FPD10, b.RISK_FPD30, b.RISK_FSTQPD30, b.client_inr_income,
# b.PRODUCT_GROUP,case when score_cb<300 then 'NTC' else 'Others' end  as NTC_FLAG,  
# row_number() over (partition by a.skp_credit_case order by a.DATE_decision asc) as rnk
# from AP_BI.bd_geo a 
# join  
# (select * from
# uw_daily_data_all 
# where time_decision_date >= trunc(sysdate)-500
# )b
# on a.skp_credit_case = b.skp_credit_case 
# where a.date_decision >= trunc(sysdate)-500
# and a.latitude is not null 
# and a.longitude is not null 
# and a.result = 'FOUND'
# and lower(DECISION_OVERALL) = 'approved'
# ) a
# where rnk = 1
# ;

In [ ]:
df= pd.read_pickle(os.getcwd()+r"\Data Files\prod_delinq_comb.pkl") #Read the data from query/downloaded file
df["DATE_DECISION"] = pd.to_datetime(df.DATE_DECISION, format = '%d-%m-%y')

### List of scores that need to be calculated

In [12]:
selected_list =['RISK_FPD10_SCORE','RISK_FPD30_SCORE','RISK_FSTQPD30_SCORE','RISK_FPD10_MM_SCORE',
'DEFAULT_FLAG_CHANGE_1','NO_PAYMENT_CURRENT_PAYMENT_1','NO_PAYMENT_CURRENT_PAYMENT_2']

### Read Hyperparameter file

In [4]:
hyper_df= pd.read_csv(os.getcwd()+r"\Data Files\Hyperparameters\Hyperparameters.csv")

### Months for which Scores need to be generated

In [14]:
df_month = pd.DataFrame({'month':[3],'year':[2020]}) #input the last month for which data is downloaded

In [16]:
df.head()

,SKP_CREDIT_CASE,DATE_DECISION,CUS_LAT,CUS_LONG,GRID_ID,RISK_FPD10,RISK_FPD30,RISK_FSTQPD30,CLIENT_INR_INCOME,PRODUCT_GROUP,NTC_FLAG
0,220372757,2019-01-01,19.375193,72.825214,500_00897_02686,0.0,0.0,0.0,20000.0,CD,NTC
1,220373060,2019-01-01,22.468687,73.285382,500_01024_03353,0.0,0.0,0.0,11000.0,CD,Others
2,220543584,2019-01-01,25.429391,81.809539,500_02737_03966,0.0,0.0,0.0,12799.4,CL,Others
3,220543586,2019-01-01,30.368656,76.846756,500_01781_05047,0.0,0.0,0.0,12000.0,CD,NTC
4,220543587,2019-01-01,17.707650,83.296908,500_03069_02293,0.0,0.0,0.0,30000.0,CD,Others


### Download all active Grids

In [ ]:
# select grid_id, avg(cus_lat), avg(cus_long) from bd_geo_delinquency
# group by grid_id
# ;

In [17]:
all_grids= pd.read_csv(os.getcwd()+r"\Data Files\grid id lat long.csv")
all_grids.columns = ["GRID_ID","CUS_LAT","CUS_LONG"]

### Generate predictors

In [18]:
df_final_test = pd.DataFrame()
df_final_grid = pd.DataFrame()

In [19]:
import datetime
from sklearn.neighbors import KNeighborsClassifier

for t in df_month.index:
    x=df_month["month"][t]
    year=df_month["year"][t]
    date_test_start = datetime.datetime(year, x, 1)
    date_test_end = (date_test_start +  datetime.timedelta(days=31)).replace(day=1)
    df_test = df[(df["DATE_DECISION"]>=date_test_start) & 
                      (df["DATE_DECISION"]<date_test_end)]
    df_grid = all_grids
    df_grid["MONTH"] = date_test_start

    for y in hyper_df.index:
        if hyper_df.loc[y,'SCORE_NAME'] in selected_list:
            score_name = hyper_df.loc[y,'SCORE_NAME']
            target = hyper_df.loc[y,'TARGET']
            K = hyper_df.loc[y,'K']
            observation_window = hyper_df.loc[y,'OBSERVATION_WINDOW']
            performance_window = hyper_df.loc[y,'PERFORMANCE_WINDOW']
            downsample_size = hyper_df.loc[y,'DOWNSAMPLE_SIZE']
            fillna = hyper_df.loc[y,'FILLNA']

            date_train_start = (date_test_start-datetime.timedelta(days=int(performance_window))-datetime.timedelta(days=int(observation_window))) 
            date_train_end = (date_test_start-datetime.timedelta(days=int(performance_window)))
            start_time = datetime.datetime.now()
            df_train = df[(df["DATE_DECISION"]>=date_train_start) & 
                      (df["DATE_DECISION"]<date_train_end)]

            df_class_0 = df_train[df_train[target] == 0]
            df_class_1 = df_train[df_train[target] == 1]
            df_class_0_under = df_class_0.sample(min(downsample_size,df_class_1.shape[0]))
            df_class_1_under = df_class_1.sample(min(downsample_size,df_class_1.shape[0]))
            df_train_under = pd.concat([df_class_0_under, df_class_1_under], axis=0)

            X_train = df_train_under[["CUS_LAT","CUS_LONG"]]
            X_test = df_test[["CUS_LAT","CUS_LONG"]]
            y_train = df_train_under[target]
            y_test =  df_test[target]

            knn = KNeighborsClassifier(n_neighbors = K)
            bst = knn.fit(X_train[["CUS_LAT","CUS_LONG"]], np.ravel(y_train))
            
            all_grids1 = all_grids
            included_grids = pd.DataFrame(df_train_under["GRID_ID"].unique())
            included_grids.columns = ['GRID_ID']
            all_grids1 = all_grids1.merge(included_grids, left_on = 'GRID_ID',right_on = 'GRID_ID', how = 'left',indicator = True)
            missing_grids=all_grids1[all_grids1["_merge"] != 'both'][["GRID_ID","CUS_LAT","CUS_LONG"]]
            df_train_under = pd.concat([df_train_under,missing_grids])

            ypred_train = bst.predict_proba(df_train_under[["CUS_LAT","CUS_LONG"]])[:, 1]
            ypred_train1 = bst.predict_proba(X_train[["CUS_LAT","CUS_LONG"]])[:, 1]  #Only for roc auc
            
            ypred_test = bst.predict_proba(X_test[["CUS_LAT","CUS_LONG"]])[:, 1]
            metric_auc_train = metrics.roc_auc_score(y_train, ypred_train1)
#            metric_auc_test = metrics.roc_auc_score(y_test, ypred_test)
            gini_train = 2 * metric_auc_train - 1
#            gini_test = 2 * metric_auc_test - 1

            df_test[score_name] = ypred_test
            
            df_train_under ["G_"+score_name] = ypred_train
            grid_score = df_train_under[["GRID_ID","G_"+score_name]].groupby("GRID_ID").mean()["G_"+score_name].reset_index()
            grid_score.columns = ['GRID_ID',"G_"+score_name]
            
            df_test = pd.merge(df_test, grid_score, on = 'GRID_ID', how = 'left')
            df_test["G_"+score_name]=df_test["G_"+score_name].fillna(value = fillna)
            
            df_grid =  pd.merge(df_grid, grid_score, on = 'GRID_ID', how = 'left')
            
            print("Training for predicting (month):",date_test_start.month, 
                  " SCORE:",score_name,
                  " target:",target,
                  " K:",K,
                  " observation_window:",observation_window,
                  " performance_window:",performance_window,
                  " downsample_size:",downsample_size,
                  " GINI TRAIN:",'{:,.2%}'.format(gini_train), 
#                   "    GINI TEST:",'{:,.2%}'.format(gini_test),
                  " time taken: ",'{:,.2}'.format((datetime.datetime.now()-start_time).seconds/60))


    if df_final_test.shape == (0,0):
        df_final_test = df_test
    else:
        df_final_test = pd.concat([df_final_test,df_test],ignore_index=True)

    if df_final_grid.shape == (0,0):
        df_final_grid = df_grid
    else:
        df_final_grid = pd.concat([df_final_grid,df_grid],ignore_index=True)


C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Training for predicting (month): 3  SCORE: RISK_FPD10_SCORE  target: RISK_FPD10  K: 271  observation_window: 365  performance_window: 40  downsample_size: 150000  GINI TRAIN: 15.55%  time taken:  2.7
Training for predicting (month): 3  SCORE: RISK_FPD30_SCORE  target: RISK_FPD30  K: 136  observation_window: 365  performance_window: 60  downsample_size: 50000  GINI TRAIN: 20.67%  time taken:  0.8
Training for predicting (month): 3  SCORE: RISK_FSTQPD30_SCORE  target: RISK_FSTQPD30  K: 376  observation_window: 365  performance_window: 150  downsample_size: 300000  GINI TRAIN: 14.62%  time taken:  6.5
Training for predicting (month): 3  SCORE: RISK_FPD10_MM_SCORE  target: RISK_FPD10  K: 241  observation_window: 110  performance_window: 40  downsample_size: 50000  GINI TRAIN: 16.08%  time taken:  1.3


In [20]:
df_final_test.to_pickle(os.getcwd()+r"\Data Files\prod_scores1.pkl")
df_final_grid.to_pickle(os.getcwd()+r"\Data Files\prod_grid_scores1.pkl")

### Download Data

In [ ]:
# drop table bd_recent_delinquency;
# create table bd_recent_delinquency as
# select date_balance, cus_lat, cus_long, grid_id, DEFAULT_FLAG_CHANGE, NO_PAYMENT_CURRENT_PAYMENT
# from
# (
# select date_balance, cus_lat, cus_long, grid_id, 
# DEFAULT_FLAG_CHANGE, NO_PAYMENT_CURRENT_PAYMENT,
# row_number() over (partition by trunc(date_balance,'mon'), 
# DEFAULT_FLAG_CHANGE, NO_PAYMENT_CURRENT_PAYMENT order by DBMS_RANDOM.VALUE) as rnk
# from
# (
# select 
# a.skp_credit_case, 
# a.skp_credit_type,
# a.date_decision,
# date_balance,
# latitude cus_lat,
# longitude cus_long, 
# grid_id, 
# dpd_current,
# dpd_earlier, 
# case when dpd_current>0 and dpd_earlier <= 0 then 1 else 0 end as default_flag_change,
# case when dpd_current - dpd_earlier > 27 then 1 else 0 end as no_payment_current_payment
# from
# (
# select skp_credit_case, skp_credit_type,
# date_decision,
# date_balance,
# cnt_days_past_due_tolerance dpd_current, 
# lag(cnt_days_past_due_tolerance,1) over (partition by skp_credit_case order by date_balance asc) dpd_earlier
# from owner_dwh.f_account_balance_pos_cash_pm
# where date_balance >= trunc(sysdate)-730
# ) a 
# join 
# AP_BI.bd_geo b 
# on a.skp_credit_case = b.skp_credit_case
# where latitude is not null
# and longitude is not null
# and grid_id is not null
# and b.result = 'FOUND'
# )
# where date_balance >= trunc(sysdate)-730
# )
# where rnk <= 300000
# ;


In [22]:
df1 = pd.read_pickle(os.getcwd()+r"\Data Files\prod_recent_delinq_comb.pkl")

In [23]:
df1["DATE_BALANCE"] = pd.to_datetime(df1.DATE_BALANCE, format = '%d-%m-%y')

In [25]:
hyper_df= pd.read_csv(os.getcwd()+r"\Data Files\Hyperparameters\Hyperparameters_EMI_Payment_Trend.csv")

In [26]:
df_final_test = pd.DataFrame()
df_final_grid = pd.DataFrame()

In [27]:


import datetime
from sklearn.neighbors import KNeighborsClassifier

for t in df_month.index:
    x=df_month["month"][t]
    year=df_month["year"][t]
    date_test_start = datetime.datetime(year, x, 1)
    date_test_end = (date_test_start +  datetime.timedelta(days=31)).replace(day=1)
    df_test = df[(df["DATE_DECISION"]>=date_test_start) & 
                      (df["DATE_DECISION"]<date_test_end)]
    df_grid = all_grids
    df_grid["MONTH"] = date_test_start

    for y in hyper_df.index:
#         if hyper_df.loc[y,'MONTH_HIST'] < x or year == 2020:
            if hyper_df.loc[y,'SCORE_NAME'] in selected_list:
                score_name = hyper_df.loc[y,'SCORE_NAME']
                target = hyper_df.loc[y,'TARGET']
                K = hyper_df.loc[y,'K']
                observation_window = hyper_df.loc[y,'OBSERVATION_WINDOW']
                performance_window = hyper_df.loc[y,'PERFORMANCE_WINDOW']
                downsample_size = hyper_df.loc[y,'DOWNSAMPLE_SIZE']

                date_train_start = (date_test_start-datetime.timedelta(days=int(performance_window)-1)-datetime.timedelta(days=int(observation_window))).replace(day=1) 
                date_train_end = (date_test_start-datetime.timedelta(days=int(performance_window)-2)).replace(day=1)
                start_time = datetime.datetime.now()
                df_train = df1[(df1["DATE_BALANCE"]>=date_train_start) & 
                          (df1["DATE_BALANCE"]<date_train_end)]

                df_class_0 = df_train[df_train[target] == 0]
                df_class_1 = df_train[df_train[target] == 1]
                df_class_0_under = df_class_0.sample(min(downsample_size,df_class_1.shape[0]))
                df_class_1_under = df_class_1.sample(min(downsample_size,df_class_1.shape[0]))
                df_train_under = pd.concat([df_class_0_under, df_class_1_under], axis=0)

                X_train = df_train_under[["CUS_LAT","CUS_LONG"]]
                X_test = df_test[["CUS_LAT","CUS_LONG"]]
                y_train = df_train_under[target]
                y_test =  df_test['RISK_FSTQPD30']

                knn = KNeighborsClassifier(n_neighbors = K)
                bst = knn.fit(X_train[["CUS_LAT","CUS_LONG"]], np.ravel(y_train))

                
                
                all_grids1 = all_grids
                included_grids = pd.DataFrame(df_train_under["GRID_ID"].unique())
                included_grids.columns = ['GRID_ID']
                all_grids1 = all_grids1.merge(included_grids, left_on = 'GRID_ID',right_on = 'GRID_ID', how = 'left',indicator = True)
                missing_grids=all_grids1[all_grids1["_merge"] != 'both'][["GRID_ID","CUS_LAT","CUS_LONG"]]
                df_train_under = pd.concat([df_train_under,missing_grids])

                ypred_train = bst.predict_proba(df_train_under[["CUS_LAT","CUS_LONG"]])[:, 1]
                ypred_train1 = bst.predict_proba(X_train[["CUS_LAT","CUS_LONG"]])[:, 1]  #Only for roc auc

                ypred_test = bst.predict_proba(X_test[["CUS_LAT","CUS_LONG"]])[:, 1]
                metric_auc_train = metrics.roc_auc_score(y_train, ypred_train1)

#                 metric_auc_test = metrics.roc_auc_score(y_test, ypred_test)
                gini_train = 2 * metric_auc_train - 1
#                 gini_test = 2 * metric_auc_test - 1

                df_test[score_name] = ypred_test
                df_train_under ["G_"+score_name] = ypred_train
                grid_score = df_train_under[["GRID_ID","G_"+score_name]].groupby("GRID_ID").mean()["G_"+score_name].reset_index()
                grid_score.columns = ['GRID_ID',"G_"+score_name]

                df_test = pd.merge(df_test, grid_score, on = 'GRID_ID', how = 'left')
                df_test["G_"+score_name]=df_test["G_"+score_name].fillna(value = fillna)
                df_grid =  pd.merge(df_grid, grid_score, on = 'GRID_ID', how = 'left')


                print("Training for predicting (month):",date_test_start.month, 
                      " SCORE:",score_name,
                      " target:",target,
                      " K:",K,
                      " observation_window:",observation_window,
                      " performance_window:",performance_window,
                      " downsample_size:",downsample_size,
                      " GINI TRAIN:",'{:,.2%}'.format(gini_train),
#                       "    GINI TEST:",'{:,.2%}'.format(gini_test),
                      " time taken: ",'{:,.2}'.format((datetime.datetime.now()-start_time).seconds/60))
#         else:
#             print("History not available")


    if df_final_test.shape == (0,0):
        df_final_test = df_test
    else:
        df_final_test = pd.concat([df_final_test,df_test],ignore_index=True)
        
    
    if df_final_grid.shape == (0,0):
        df_final_grid = df_grid
    else:
        df_final_grid = pd.concat([df_final_grid,df_grid],ignore_index=True)


C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Training for predicting (month): 3  SCORE: DEFAULT_FLAG_CHANGE_1  target: DEFAULT_FLAG_CHANGE  K: 136  observation_window: 30  performance_window: 0  downsample_size: 80000  GINI TRAIN: 20.77%  time taken:  0.92
Training for predicting (month): 3  SCORE: NO_PAYMENT_CURRENT_PAYMENT_1  target: NO_PAYMENT_CURRENT_PAYMENT  K: 351  observation_window: 30  performance_window: 0  downsample_size: 300000  GINI TRAIN: 13.16%  time taken:  3.4
Training for predicting (month): 3  SCORE: NO_PAYMENT_CURRENT_PAYMENT_2  target: NO_PAYMENT_CURRENT_PAYMENT  K: 351  observation_window: 30  performance_window: 30  downsample_size: 300000  GINI TRAIN: 11.79%  time taken:  3.4


In [122]:
df_final_test.to_pickle(os.getcwd()+r"\Data Files\prod_scores2.pkl")
df_final_grid.to_pickle(os.getcwd()+r"\Data Files\prod_grid_scores2.pkl")

In [136]:
df_grid=pd.read_pickle(os.getcwd()+r"\Data Files\prod_grid_scores1.pkl")
df1_grid= pd.read_pickle(os.getcwd()+r"\Data Files\prod_grid_scores2.pkl")

In [139]:
df1_grid = df1_grid[['GRID_ID','MONTH','G_DEFAULT_FLAG_CHANGE_1',
          'G_NO_PAYMENT_CURRENT_PAYMENT_1','G_NO_PAYMENT_CURRENT_PAYMENT_2']]
df_grid= df_grid.merge(df1_grid, on = ['GRID_ID','MONTH'],how = 'right')
df_grid.shape

(2530785, 11)

In [144]:
scores_columns= [ 'G_RISK_FPD10_SCORE','G_RISK_FPD30_SCORE','G_RISK_FSTQPD30_SCORE','G_RISK_FPD10_MM_SCORE','G_DEFAULT_FLAG_CHANGE_1',
                 'G_NO_PAYMENT_CURRENT_PAYMENT_1','G_NO_PAYMENT_CURRENT_PAYMENT_2']

### Convert All Scores to Logit

In [162]:
for y in scores_columns:
    df_grid["LOGIT_"+y] = [np.log(x/(1-x)) for x in df_grid[y]]
#     df["LOGIT_"+y] = np.log(df[y]/(1-df[y]))

In [163]:
logit_columns= [x for x in df_grid.columns if 'LOGIT' in x]

In [164]:
df1=df_grid

In [167]:
df1["COMBINED_SCORE_KNN_EXI"] = 1/(1+np.exp(-(df1["LOGIT_G_RISK_FPD30_SCORE"]*0.133694+ 
              df1["LOGIT_G_RISK_FSTQPD30_SCORE"]*0.253384 +
              df1["LOGIT_G_DEFAULT_FLAG_CHANGE_1"]*0.138207 +
              df1["LOGIT_G_NO_PAYMENT_CURRENT_PAYMENT_2"]*0.298317 +
              df1["LOGIT_G_RISK_FPD10_MM_SCORE"]*0.189856 +
              df1["LOGIT_G_RISK_FPD10_SCORE"]*0.163698 +  -2.7974444)))
df1["COMBINED_SCORE_KNN_NTC"] =1/(1+np.exp(-(df1["LOGIT_G_RISK_FSTQPD30_SCORE"]*0.495134+ 
              df1["LOGIT_G_NO_PAYMENT_CURRENT_PAYMENT_1"]*0.246826 +
                df1["LOGIT_G_DEFAULT_FLAG_CHANGE_1"]*0.128352 +
              df1["LOGIT_G_RISK_FPD10_MM_SCORE"]*0.123364 +  -2.37891549)))

In [169]:
df1.to_csv(os.getcwd()+r"\Data Files\prod_grid_scored.csv", index = False)

### Download data for Upload

In [172]:
df1[["GRID_ID","MONTH","COMBINED_SCORE_KNN_EXI","COMBINED_SCORE_KNN_NTC"]].to_csv(os.getcwd()+r"\Data Files\prod_grid_scored_upload.csv", index = False)